In [1]:
import numpy as np

import tensorflow as tf

In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM, Reshape, Activation, Input
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K


In [3]:
# from keras.preprocessing import sequence
# from keras.models import Sequential, Model
# from keras.layers import Dense, Embedding
# from keras.layers import LSTM, Reshape, Activation, Input
# from keras.datasets import imdb
# from keras.utils import to_categorical
# import keras.backend as K

In [4]:
# import keras.backend as KB

In [5]:
# import theano
# import theano.tensor as T

In [6]:
%%time
max_features = 2500
# cut texts after this number of words (among top max_features most common words)
maxlen = 80

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features, seed=1)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences
CPU times: user 4.22 s, sys: 299 ms, total: 4.52 s
Wall time: 4.64 s


In [7]:
lens = np.array([len(x) for x in x_train])

print("Average number of words in each review:", lens.mean())

Average number of words in each review: 238.71364


In [8]:
print("One-hot encoding of labels")
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)
y_train.shape

One-hot encoding of labels


(25000, 2)

In [9]:
y_test.shape

(25000, 2)

In [10]:
maxlen = 80

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [11]:
# need to some setup so everything gets excecuted in the same tensorflow session
# myGraph = tf.Graph()
# session = tf.Session(graph=myGraph )
K.clear_session()
# K.set_session( session )


In [12]:
sentence = Input(shape=(80,))
embeddings = Embedding(max_features, 128)(sentence)
lstm_out = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embeddings)
dense_out = Dense(2)(lstm_out)
out = Activation('softmax')(dense_out)

In [14]:
imdb_clf = Model(inputs=sentence, outputs=out)
imdb_clf.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
imdb_clf.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 80)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 80, 128)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
_________________________________________________________________
activation (Activation)      (None, 2)                 0         
Total params: 369,538
Trainable params: 369,538
Non-trainable params: 0
_________________________________________________________________


In [15]:
# imdb_clf._layers.pop(0)
# imdb_clf._layers.pop(0)
# imdb_clf.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# imdb_clf.summary()

In [16]:
# print('Build model...')
# imdb_classifier = Sequential()
# imdb_classifier.add(Embedding(max_features, 128, input_length=maxlen))
# imdb_classifier.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
# # imdb_classifier.add(Reshape(target_shape=(1,128,1)))
# # imdb_classifier.add(Dense(2, activation='linear'))
# # imdb_classifier.add(TimeDistributed(Dense(2)))
# # imdb_classifier.add(Reshape(target_shape=(1,2)))
# imdb_classifier.add(Dense(2))
# imdb_classifier.add(Activation('softmax'))

# # try using different optimizers and different optimizer configs
# imdb_classifier.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
          
# imdb_classifier.summary()

In [17]:
batch_size = 256

print('Train...')
imdb_clf.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(x_test, y_test))

Train...
Train on 25000 samples, validate on 25000 samples
25000/25000 [==============================] - 25s 992us/sample - loss: 0.5342 - accuracy: 0.7289 - val_loss: 0.3955 - val_accuracy: 0.8224


In [18]:
# imdb_clf.trainable_variables[0]

In [19]:
# sentence = Input(shape=(80,))
# embeddings = Embedding(max_features, 128)(sentence)

# generate_embeddings = Model(inputs=imdb_clf.inputs, outputs=imdb_clf.layers[1].output)
# generate_embeddings.predict(x_test[0:5])

In [20]:
# imdb_clf._layers.pop(0)
# imdb_clf._layers.pop(0)
# imdb_clf.summary()

In [23]:
embed_input = Input(shape=layer_output[0].shape)
embed_lstm = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embed_input)
embed_dense = Dense(2)(embed_lstm)

embed_model = Model(inputs=embed_input, outputs=embed_dense)
embed_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 80, 128)]         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 49,538
Trainable params: 49,538
Non-trainable params: 0
_________________________________________________________________


In [24]:
get_embeddings = K.function([imdb_clf.layers[0].input],
                                  imdb_clf.layers[1].output)
layer_output = get_embeddings(x_test[0:5])
layer_output.shape

(5, 80, 128)

In [ ]:
vocab_embeddings = imdb_clf.layers[1].embeddings.numpy()

In [ ]:
embed_model.layers[1].set_weights(imdb_clf.layers[2].get_weights())
embed_model.layers[2].set_weights(imdb_clf.layers[3].get_weights())

In [ ]:
# grad_model = Model(inputs=imdb_clf.inputs,outputs=imdb_clf.layers[-2].output)

In [ ]:
# # an input layer to feed labels
# y_true = Input(shape=(2,))
# # compute loss based on model's output and true labels
# ce = K.categorical_crossentropy(y_true, imdb_clf.output)
# # compute gradient of loss with respect to inputs
# grad_ce = K.gradients(ce, imdb_clf.inputs)
# # create a function to be able to run this computation graph
# func = K.function(imdb_clf.inputs + [y_true], grad_ce)

# # usage
# output = func([sample, real_y])

In [ ]:
# sample = x_test[0:5]
# real_y = y_test[0:5]

In [ ]:
# output_layer = imdb_clf.outputs
# all_layers = [layer.output for layer in imdb_clf.layers]
# new_model = Model(inputs=imdb_clf.inputs, outputs=all_layers)

# sample = tf.convert_to_tensor(x_test[0:5], tf.float32)
# real_y = y_test[0:5]

# samples = tf.Variable(sample, dtype=tf.float32)

# # inputs = tf.ones((1, 299, 299, 3))
# with tf.GradientTape() as tape:
#     output_of_all_layers = new_model(samples)
#     preds = output_layer[-1]  # last layer is output layer
#     print(preds)
#     # take gradients of last layer with respect to all layers in the model
#     grads = tape.gradient(preds, output_of_all_layers)
#     # note: grads[-1] should be all 1, since it it d(output)/d(output)
# print(grads)

In [ ]:
sample = tf.convert_to_tensor(layer_output[0:5].reshape(-1,80,128), tf.float32)
real_y = y_test[0:5]

samples = tf.Variable(sample, dtype=tf.float32)
with tf.GradientTape(watch_accessed_variables=False) as tape:
#     tape.reset()
    tape.watch(samples)
#     samples = tf.Variable(sample, dtype=tf.float32)

    print("step1")
    # Make prediction
    pred_y = embed_model(samples)
    print("step2")
    # Calculate loss
    model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    print("step3", model_loss)
    print()
    #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)

# print(tape.watched_variables())
# Calculate gradients
model_gradients = tape.batch_jacobian(pred_y, samples).numpy()
print(model_gradients.reshape(-1,2,80,128).shape)
# Update model
# optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
print(model_gradients.reshape(-1,2,80,128).shape)

In [ ]:
score, acc = imdb_classifier.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
get_logits = K.function([imdb_clf.layers[0].input],[imdb_clf.layers[-1].input])
logits = get_logits(x_test[0:5])[0]
logits

In [ ]:
grad_func = K.function([imdb_classifier.layers[1].input], [imdb_classifier.layers[-2].output])

In [ ]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_test[0:5], logits=logits)
loss

In [ ]:
# samples = x_test[0:5]
# real_y = y_test[0:5]

# samples = tf.Variable(samples, dtype=tf.float32)
# with tf.GradientTape() as tape:
#     tape.watch(samples)
#     print("step1")
#     # Make prediction
#     pred_y = grad_model(samples)
#     print("step2")
#     # Calculate loss
#     model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     print("step3")
#     print(model_loss)
#     #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    
# # Calculate gradients
# model_gradients = tape.gradient(model_loss, samples)
# print(model_gradients)
# # Update model
# # optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
# # tf.compat.v1.enable_eager_execution()
# samples = tf.convert_to_tensor(layer_output, np.float32)
# # inp = tf.Variable(layer_output, dtype=tf.float32)
# # labels = tf.convert_to_tensor(y_test[0:10], np.float32)
# labels = y_test[0:10]
# with tf.GradientTape() as tape:
# #     tape.watch(samples)
# # def fun(sample):
#     logits = grad_model(samples)[0]
#     loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# # grads = tape.gradient(loss, inp)
# # tf.test.compute_gradient(fun,[samples])
# grads = K.gradients(loss,samples)
    
# print(grads)

In [ ]:
# from cleverhans.compat import softmax_cross_entropy_with_logits
# from cleverhans.utils_keras import KerasModelWrapper

In [ ]:
# wrapper = KerasModelWrapper( model )

In [ ]:
# sample_ind = 100 # chosen by totaly random dice roll

# # picking a test sample
# sample = x_train[ sample_ind, : ]
# true_label = y_train[ sample_ind, : ]

# sample = x_train[ 0:10 ]
# true_label = y_train[ 0:10 ]

# # sample = sample.reshape(-1,80)
# # true_label = true_label.reshape(1,2)

In [ ]:
# myGraph = tf.Graph()
# with myGraph.as_default():
#     x = model.layers[ 0 ].input
#     y = tf.placeholder( tf.float32, shape=(None, 2) )
#     logits = model.layers[-1].input

#     loss = softmax_cross_entropy_with_logits(labels=y,logits=logits)

#     grads = tf.gradients(loss,x)


# init = tf.global_variables_initializer()

# session.run(init)
# output = session.run(grads, feed_dict={x: sample, y : true_label})

In [ ]:
embed_model.layers[1].set_weights(imdb_clf.layers[2].get_weights())
embed_model.layers[2].set_weights(imdb_clf.layers[3].get_weights())

In [ ]:
# grad_model = Model(inputs=imdb_clf.inputs,outputs=imdb_clf.layers[-2].output)

In [ ]:
# # an input layer to feed labels
# y_true = Input(shape=(2,))
# # compute loss based on model's output and true labels
# ce = K.categorical_crossentropy(y_true, imdb_clf.output)
# # compute gradient of loss with respect to inputs
# grad_ce = K.gradients(ce, imdb_clf.inputs)
# # create a function to be able to run this computation graph
# func = K.function(imdb_clf.inputs + [y_true], grad_ce)

# # usage
# output = func([sample, real_y])

In [ ]:
# sample = x_test[0:5]
# real_y = y_test[0:5]

In [ ]:
# output_layer = imdb_clf.outputs
# all_layers = [layer.output for layer in imdb_clf.layers]
# new_model = Model(inputs=imdb_clf.inputs, outputs=all_layers)

# sample = tf.convert_to_tensor(x_test[0:5], tf.float32)
# real_y = y_test[0:5]

# samples = tf.Variable(sample, dtype=tf.float32)

# # inputs = tf.ones((1, 299, 299, 3))
# with tf.GradientTape() as tape:
#     output_of_all_layers = new_model(samples)
#     preds = output_layer[-1]  # last layer is output layer
#     print(preds)
#     # take gradients of last layer with respect to all layers in the model
#     grads = tape.gradient(preds, output_of_all_layers)
#     # note: grads[-1] should be all 1, since it it d(output)/d(output)
# print(grads)

In [ ]:
sample = tf.convert_to_tensor(layer_output[0:5].reshape(-1,80,128), tf.float32)
real_y = y_test[0:5]

samples = tf.Variable(sample, dtype=tf.float32)
with tf.GradientTape(watch_accessed_variables=False) as tape:
#     tape.reset()
    tape.watch(samples)
#     samples = tf.Variable(sample, dtype=tf.float32)

    print("step1")
    # Make prediction
    pred_y = embed_model(samples)
    print("step2")
    # Calculate loss
    model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    print("step3", model_loss)
    print()
    #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)

# print(tape.watched_variables())
# Calculate gradients
model_gradients = tape.batch_jacobian(pred_y, samples).numpy()
print(model_gradients.reshape(-1,2,80,128).shape)
# Update model
# optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
print(model_gradients.reshape(-1,2,80,128).shape)

In [ ]:
score, acc = imdb_classifier.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
get_logits = K.function([imdb_clf.layers[0].input],[imdb_clf.layers[-1].input])
logits = get_logits(x_test[0:5])[0]
logits

In [ ]:
grad_func = K.function([imdb_classifier.layers[1].input], [imdb_classifier.layers[-2].output])

In [ ]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_test[0:5], logits=logits)
loss

In [ ]:
# samples = x_test[0:5]
# real_y = y_test[0:5]

# samples = tf.Variable(samples, dtype=tf.float32)
# with tf.GradientTape() as tape:
#     tape.watch(samples)
#     print("step1")
#     # Make prediction
#     pred_y = grad_model(samples)
#     print("step2")
#     # Calculate loss
#     model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     print("step3")
#     print(model_loss)
#     #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    
# # Calculate gradients
# model_gradients = tape.gradient(model_loss, samples)
# print(model_gradients)
# # Update model
# # optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
# # tf.compat.v1.enable_eager_execution()
# samples = tf.convert_to_tensor(layer_output, np.float32)
# # inp = tf.Variable(layer_output, dtype=tf.float32)
# # labels = tf.convert_to_tensor(y_test[0:10], np.float32)
# labels = y_test[0:10]
# with tf.GradientTape() as tape:
# #     tape.watch(samples)
# # def fun(sample):
#     logits = grad_model(samples)[0]
#     loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# # grads = tape.gradient(loss, inp)
# # tf.test.compute_gradient(fun,[samples])
# grads = K.gradients(loss,samples)
    
# print(grads)

In [ ]:
# from cleverhans.compat import softmax_cross_entropy_with_logits
# from cleverhans.utils_keras import KerasModelWrapper

In [ ]:
# wrapper = KerasModelWrapper( model )

In [ ]:
# sample_ind = 100 # chosen by totaly random dice roll

# # picking a test sample
# sample = x_train[ sample_ind, : ]
# true_label = y_train[ sample_ind, : ]

# sample = x_train[ 0:10 ]
# true_label = y_train[ 0:10 ]

# # sample = sample.reshape(-1,80)
# # true_label = true_label.reshape(1,2)

In [ ]:
# myGraph = tf.Graph()
# with myGraph.as_default():
#     x = model.layers[ 0 ].input
#     y = tf.placeholder( tf.float32, shape=(None, 2) )
#     logits = model.layers[-1].input

#     loss = softmax_cross_entropy_with_logits(labels=y,logits=logits)

#     grads = tf.gradients(loss,x)


# init = tf.global_variables_initializer()

# session.run(init)
# output = session.run(grads, feed_dict={x: sample, y : true_label})

In [ ]:
embed_model.layers[1].set_weights(imdb_clf.layers[2].get_weights())
embed_model.layers[2].set_weights(imdb_clf.layers[3].get_weights())

In [ ]:
# grad_model = Model(inputs=imdb_clf.inputs,outputs=imdb_clf.layers[-2].output)

In [ ]:
# # an input layer to feed labels
# y_true = Input(shape=(2,))
# # compute loss based on model's output and true labels
# ce = K.categorical_crossentropy(y_true, imdb_clf.output)
# # compute gradient of loss with respect to inputs
# grad_ce = K.gradients(ce, imdb_clf.inputs)
# # create a function to be able to run this computation graph
# func = K.function(imdb_clf.inputs + [y_true], grad_ce)

# # usage
# output = func([sample, real_y])

In [ ]:
# sample = x_test[0:5]
# real_y = y_test[0:5]

In [ ]:
# output_layer = imdb_clf.outputs
# all_layers = [layer.output for layer in imdb_clf.layers]
# new_model = Model(inputs=imdb_clf.inputs, outputs=all_layers)

# sample = tf.convert_to_tensor(x_test[0:5], tf.float32)
# real_y = y_test[0:5]

# samples = tf.Variable(sample, dtype=tf.float32)

# # inputs = tf.ones((1, 299, 299, 3))
# with tf.GradientTape() as tape:
#     output_of_all_layers = new_model(samples)
#     preds = output_layer[-1]  # last layer is output layer
#     print(preds)
#     # take gradients of last layer with respect to all layers in the model
#     grads = tape.gradient(preds, output_of_all_layers)
#     # note: grads[-1] should be all 1, since it it d(output)/d(output)
# print(grads)

In [ ]:
sample = tf.convert_to_tensor(layer_output[0:5].reshape(-1,80,128), tf.float32)
real_y = y_test[0:5]

samples = tf.Variable(sample, dtype=tf.float32)
with tf.GradientTape(watch_accessed_variables=False) as tape:
#     tape.reset()
    tape.watch(samples)
#     samples = tf.Variable(sample, dtype=tf.float32)

    print("step1")
    # Make prediction
    pred_y = embed_model(samples)
    print("step2")
    # Calculate loss
    model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    print("step3", model_loss)
    print()
    #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)

# print(tape.watched_variables())
# Calculate gradients
model_gradients = tape.batch_jacobian(pred_y, samples).numpy()
print(model_gradients.reshape(-1,2,80,128).shape)
# Update model
# optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
print(model_gradients.reshape(-1,2,80,128).shape)

In [ ]:
score, acc = imdb_classifier.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
get_logits = K.function([imdb_clf.layers[0].input],[imdb_clf.layers[-1].input])
logits = get_logits(x_test[0:5])[0]
logits

In [ ]:
grad_func = K.function([imdb_classifier.layers[1].input], [imdb_classifier.layers[-2].output])

In [ ]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_test[0:5], logits=logits)
loss

In [ ]:
# samples = x_test[0:5]
# real_y = y_test[0:5]

# samples = tf.Variable(samples, dtype=tf.float32)
# with tf.GradientTape() as tape:
#     tape.watch(samples)
#     print("step1")
#     # Make prediction
#     pred_y = grad_model(samples)
#     print("step2")
#     # Calculate loss
#     model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     print("step3")
#     print(model_loss)
#     #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    
# # Calculate gradients
# model_gradients = tape.gradient(model_loss, samples)
# print(model_gradients)
# # Update model
# # optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
# # tf.compat.v1.enable_eager_execution()
# samples = tf.convert_to_tensor(layer_output, np.float32)
# # inp = tf.Variable(layer_output, dtype=tf.float32)
# # labels = tf.convert_to_tensor(y_test[0:10], np.float32)
# labels = y_test[0:10]
# with tf.GradientTape() as tape:
# #     tape.watch(samples)
# # def fun(sample):
#     logits = grad_model(samples)[0]
#     loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# # grads = tape.gradient(loss, inp)
# # tf.test.compute_gradient(fun,[samples])
# grads = K.gradients(loss,samples)
    
# print(grads)

In [ ]:
# from cleverhans.compat import softmax_cross_entropy_with_logits
# from cleverhans.utils_keras import KerasModelWrapper

In [ ]:
# wrapper = KerasModelWrapper( model )

In [ ]:
# sample_ind = 100 # chosen by totaly random dice roll

# # picking a test sample
# sample = x_train[ sample_ind, : ]
# true_label = y_train[ sample_ind, : ]

# sample = x_train[ 0:10 ]
# true_label = y_train[ 0:10 ]

# # sample = sample.reshape(-1,80)
# # true_label = true_label.reshape(1,2)

In [ ]:
# myGraph = tf.Graph()
# with myGraph.as_default():
#     x = model.layers[ 0 ].input
#     y = tf.placeholder( tf.float32, shape=(None, 2) )
#     logits = model.layers[-1].input

#     loss = softmax_cross_entropy_with_logits(labels=y,logits=logits)

#     grads = tf.gradients(loss,x)


# init = tf.global_variables_initializer()

# session.run(init)
# output = session.run(grads, feed_dict={x: sample, y : true_label})

In [ ]:
embed_model.layers[1].set_weights(imdb_clf.layers[2].get_weights())
embed_model.layers[2].set_weights(imdb_clf.layers[3].get_weights())

In [ ]:
# grad_model = Model(inputs=imdb_clf.inputs,outputs=imdb_clf.layers[-2].output)

In [ ]:
# # an input layer to feed labels
# y_true = Input(shape=(2,))
# # compute loss based on model's output and true labels
# ce = K.categorical_crossentropy(y_true, imdb_clf.output)
# # compute gradient of loss with respect to inputs
# grad_ce = K.gradients(ce, imdb_clf.inputs)
# # create a function to be able to run this computation graph
# func = K.function(imdb_clf.inputs + [y_true], grad_ce)

# # usage
# output = func([sample, real_y])

In [ ]:
# sample = x_test[0:5]
# real_y = y_test[0:5]

In [ ]:
# output_layer = imdb_clf.outputs
# all_layers = [layer.output for layer in imdb_clf.layers]
# new_model = Model(inputs=imdb_clf.inputs, outputs=all_layers)

# sample = tf.convert_to_tensor(x_test[0:5], tf.float32)
# real_y = y_test[0:5]

# samples = tf.Variable(sample, dtype=tf.float32)

# # inputs = tf.ones((1, 299, 299, 3))
# with tf.GradientTape() as tape:
#     output_of_all_layers = new_model(samples)
#     preds = output_layer[-1]  # last layer is output layer
#     print(preds)
#     # take gradients of last layer with respect to all layers in the model
#     grads = tape.gradient(preds, output_of_all_layers)
#     # note: grads[-1] should be all 1, since it it d(output)/d(output)
# print(grads)

In [ ]:
sample = tf.convert_to_tensor(layer_output[0:5].reshape(-1,80,128), tf.float32)
real_y = y_test[0:5]

samples = tf.Variable(sample, dtype=tf.float32)
with tf.GradientTape(watch_accessed_variables=False) as tape:
#     tape.reset()
    tape.watch(samples)
#     samples = tf.Variable(sample, dtype=tf.float32)

    print("step1")
    # Make prediction
    pred_y = embed_model(samples)
    print("step2")
    # Calculate loss
    model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    print("step3", model_loss)
    print()
    #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)

# print(tape.watched_variables())
# Calculate gradients
model_gradients = tape.batch_jacobian(pred_y, samples).numpy()
print(model_gradients.reshape(-1,2,80,128).shape)
# Update model
# optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
print(model_gradients.reshape(-1,2,80,128).shape)

In [ ]:
score, acc = imdb_classifier.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
get_logits = K.function([imdb_clf.layers[0].input],[imdb_clf.layers[-1].input])
logits = get_logits(x_test[0:5])[0]
logits

In [ ]:
grad_func = K.function([imdb_classifier.layers[1].input], [imdb_classifier.layers[-2].output])

In [ ]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_test[0:5], logits=logits)
loss

In [ ]:
# samples = x_test[0:5]
# real_y = y_test[0:5]

# samples = tf.Variable(samples, dtype=tf.float32)
# with tf.GradientTape() as tape:
#     tape.watch(samples)
#     print("step1")
#     # Make prediction
#     pred_y = grad_model(samples)
#     print("step2")
#     # Calculate loss
#     model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     print("step3")
#     print(model_loss)
#     #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    
# # Calculate gradients
# model_gradients = tape.gradient(model_loss, samples)
# print(model_gradients)
# # Update model
# # optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [ ]:
# # tf.compat.v1.enable_eager_execution()
# samples = tf.convert_to_tensor(layer_output, np.float32)
# # inp = tf.Variable(layer_output, dtype=tf.float32)
# # labels = tf.convert_to_tensor(y_test[0:10], np.float32)
# labels = y_test[0:10]
# with tf.GradientTape() as tape:
# #     tape.watch(samples)
# # def fun(sample):
#     logits = grad_model(samples)[0]
#     loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# # grads = tape.gradient(loss, inp)
# # tf.test.compute_gradient(fun,[samples])
# grads = K.gradients(loss,samples)
    
# print(grads)

In [ ]:
# from cleverhans.compat import softmax_cross_entropy_with_logits
# from cleverhans.utils_keras import KerasModelWrapper

In [ ]:
# wrapper = KerasModelWrapper( model )

In [ ]:
# sample_ind = 100 # chosen by totaly random dice roll

# # picking a test sample
# sample = x_train[ sample_ind, : ]
# true_label = y_train[ sample_ind, : ]

# sample = x_train[ 0:10 ]
# true_label = y_train[ 0:10 ]

# # sample = sample.reshape(-1,80)
# # true_label = true_label.reshape(1,2)

In [ ]:
# myGraph = tf.Graph()
# with myGraph.as_default():
#     x = model.layers[ 0 ].input
#     y = tf.placeholder( tf.float32, shape=(None, 2) )
#     logits = model.layers[-1].input

#     loss = softmax_cross_entropy_with_logits(labels=y,logits=logits)

#     grads = tf.gradients(loss,x)


# init = tf.global_variables_initializer()

# session.run(init)
# output = session.run(grads, feed_dict={x: sample, y : true_label})

In [25]:
embed_model.layers[1].set_weights(imdb_clf.layers[2].get_weights())
embed_model.layers[2].set_weights(imdb_clf.layers[3].get_weights())

In [26]:
# grad_model = Model(inputs=imdb_clf.inputs,outputs=imdb_clf.layers[-2].output)

In [27]:
# # an input layer to feed labels
# y_true = Input(shape=(2,))
# # compute loss based on model's output and true labels
# ce = K.categorical_crossentropy(y_true, imdb_clf.output)
# # compute gradient of loss with respect to inputs
# grad_ce = K.gradients(ce, imdb_clf.inputs)
# # create a function to be able to run this computation graph
# func = K.function(imdb_clf.inputs + [y_true], grad_ce)

# # usage
# output = func([sample, real_y])

In [28]:
# sample = x_test[0:5]
# real_y = y_test[0:5]

In [29]:
# output_layer = imdb_clf.outputs
# all_layers = [layer.output for layer in imdb_clf.layers]
# new_model = Model(inputs=imdb_clf.inputs, outputs=all_layers)

# sample = tf.convert_to_tensor(x_test[0:5], tf.float32)
# real_y = y_test[0:5]

# samples = tf.Variable(sample, dtype=tf.float32)

# # inputs = tf.ones((1, 299, 299, 3))
# with tf.GradientTape() as tape:
#     output_of_all_layers = new_model(samples)
#     preds = output_layer[-1]  # last layer is output layer
#     print(preds)
#     # take gradients of last layer with respect to all layers in the model
#     grads = tape.gradient(preds, output_of_all_layers)
#     # note: grads[-1] should be all 1, since it it d(output)/d(output)
# print(grads)

In [45]:
sample = tf.convert_to_tensor(layer_output[0:5].reshape(-1,80,128), tf.float32)
real_y = y_test[0:5]

samples = tf.Variable(sample, dtype=tf.float32)
with tf.GradientTape(watch_accessed_variables=False) as tape:
#     tape.reset()
    tape.watch(samples)
#     samples = tf.Variable(sample, dtype=tf.float32)

    print("step1")
    # Make prediction
    pred_y = embed_model(samples)
    print("step2")
    # Calculate loss
    model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    print("step3", model_loss)
    print()
    #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)

# print(tape.watched_variables())
# Calculate gradients
model_gradients = tape.batch_jacobian(pred_y, samples).numpy()
print(model_gradients.reshape(-1,2,80,128).shape)
# Update model
# optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

step1
step2
step3 tf.Tensor([0.24223822 0.03356474 0.31229496 0.49673867 0.05070358], shape=(5,), dtype=float32)

(5, 2, 80, 128)


In [42]:
print(model_gradients.reshape(-1,2,80,128).shape)

(1, 2, 80, 128)


In [12]:
score, acc = imdb_classifier.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 14s 566us/step
Test score: 0.3708902388572693
Test accuracy: 0.8351200222969055


In [36]:
get_logits = K.function([imdb_clf.layers[0].input],[imdb_clf.layers[-1].input])
logits = get_logits(x_test[0:5])[0]
logits

array([[ 0.5548203 , -0.61283696],
       [ 1.2027019 , -1.2958295 ],
       [ 0.11605286, -0.14667611],
       [ 0.62695706, -0.6806632 ],
       [ 1.1918951 , -1.2904471 ]], dtype=float32)

In [24]:
grad_func = K.function([imdb_classifier.layers[1].input], [imdb_classifier.layers[-2].output])

In [38]:
loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_test[0:5], logits=logits)
loss

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.27086258, 0.07900123, 0.5703863 , 0.23938139, 0.08024024],
      dtype=float32)>

In [192]:
# samples = x_test[0:5]
# real_y = y_test[0:5]

# samples = tf.Variable(samples, dtype=tf.float32)
# with tf.GradientTape() as tape:
#     tape.watch(samples)
#     print("step1")
#     # Make prediction
#     pred_y = grad_model(samples)
#     print("step2")
#     # Calculate loss
#     model_loss = tf.nn.softmax_cross_entropy_with_logits(labels=real_y, logits=pred_y)
#     print("step3")
#     print(model_loss)
#     #         model_loss = tf.keras.losses.categorical_crossentropy(real_y, pred_y)
    
# # Calculate gradients
# model_gradients = tape.gradient(model_loss, samples)
# print(model_gradients)
# # Update model
# # optimizer.apply_gradients(zip(model_gradients, model.trainable_variables))

In [67]:
# # tf.compat.v1.enable_eager_execution()
# samples = tf.convert_to_tensor(layer_output, np.float32)
# # inp = tf.Variable(layer_output, dtype=tf.float32)
# # labels = tf.convert_to_tensor(y_test[0:10], np.float32)
# labels = y_test[0:10]
# with tf.GradientTape() as tape:
# #     tape.watch(samples)
# # def fun(sample):
#     logits = grad_model(samples)[0]
#     loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# # grads = tape.gradient(loss, inp)
# # tf.test.compute_gradient(fun,[samples])
# grads = K.gradients(loss,samples)
    
# print(grads)

[None]


In [190]:
# from cleverhans.compat import softmax_cross_entropy_with_logits
# from cleverhans.utils_keras import KerasModelWrapper

In [191]:
# wrapper = KerasModelWrapper( model )

In [189]:
# sample_ind = 100 # chosen by totaly random dice roll

# # picking a test sample
# sample = x_train[ sample_ind, : ]
# true_label = y_train[ sample_ind, : ]

# sample = x_train[ 0:10 ]
# true_label = y_train[ 0:10 ]

# # sample = sample.reshape(-1,80)
# # true_label = true_label.reshape(1,2)

In [188]:
# myGraph = tf.Graph()
# with myGraph.as_default():
#     x = model.layers[ 0 ].input
#     y = tf.placeholder( tf.float32, shape=(None, 2) )
#     logits = model.layers[-1].input

#     loss = softmax_cross_entropy_with_logits(labels=y,logits=logits)

#     grads = tf.gradients(loss,x)


# init = tf.global_variables_initializer()

# session.run(init)
# output = session.run(grads, feed_dict={x: sample, y : true_label})